In [1]:
# !git clone https://github.com/drminix/first-order-model

In [2]:
import imageio # 비디오 저장 (출력)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation # 동영상 인스턴스 생성

import skvideo.io # 비디오 입력

from skimage.transform import resize
from skimage import img_as_ubyte # 비디오 저장 (출력) 시 부호없는 8비트 정수로 변환

from IPython.display import HTML # 노트북 출력

import os
os.chdir('first-order-model')

# scikit-image == 0.18.0 필요
# first-order-model 스크립트
from demo import make_animation
from demo import load_checkpoints

import warnings
warnings.filterwarnings('ignore')

In [3]:
def display_func(source, driving, generated=None):
    
    fig = plt.figure(figsize=(8+4*(generated is not None),6)) # generated True 일 경우 (12, 6)
    
    img = []
    
    for idx in range(len(driving)): # Frame 수
        
        cols = [source] # (1, 256, 256, 3)
        cols.append(driving[idx]) # (2, 256, 256, 3) = [소스 이미지 Array] + [프레임 이미지 Array]
        
        if generated is not None:
            cols.append(generated[idx]) # (3, 256, 256, 3) = [소스 이미지 Array] + [프레임 이미지 Array] + [생성값 Array]
        
        # Frame 별 이미지 저장
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True) # (256, 512, 3) / generated is not None = (256, 768, 3)
        plt.axis('off')
        
        # 컨테이너에 Frame 추가
        img.append([im])
    
    # ArtistAnimation 인스턴스 반환
    animate = animation.ArtistAnimation(fig,
                                        img,
                                        interval=50,
                                        repeat_delay=1000)
    
    plt.close()
    
    return animate

In [4]:
# 입력사진
source_image = imageio.imread('../data/image/01_byunghun_01.png') # (1024, 1024, 3)

# 입력영상
driving_video = skvideo.io.vread('../data/video/04.mp4') # (211, 256, 256, 3)

# 사진 및 영상 Resize
resized_source_image = resize(source_image, (256, 256))[..., :3] # (256, 256, 3)
resized_driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video] # List[(256, 256, 3)] : 프레임별 Resize

In [5]:
HTML(display_func(resized_source_image, resized_driving_video).to_html5_video())

In [6]:
# 생성 모델 로드
generator, kp_detector = load_checkpoints(
    config_path = 'config/vox-256.yaml',
    # 모델 파일 다운로드 : https://www.kaggle.com/code/ditmcz/talking-head-video-generation/input
    checkpoint_path = '../data/model/vox-cpk.pth.tar' 
)

In [7]:
# 모델 예측 (합성 동영상 생성)
predictions = make_animation(
    resized_source_image,
    resized_driving_video,
    generator,
    kp_detector,
    relative=True)

# 합성 영상 저장
imageio.mimsave('../generated_trump.mp4', [img_as_ubyte(frame) for frame in predictions])

HTML(display_func(resized_source_image, resized_driving_video, predictions).to_html5_video())

100%|████████████████████████████████████████████████████████████████████████████████| 211/211 [00:42<00:00,  5.00it/s]


In [8]:
# 입력 사진
source_image = imageio.imread('../data/image/eminem.jpg')
source_image = resize(source_image, (256, 256))[..., :3]

# 입력 영상
driving_video = skvideo.io.vread('../data/video/damedame.mp4')
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

# 합성 비디오 생성
# 프레임 수에 따라 연산량 ↑
predictions = make_animation(
    source_image,
    driving_video,
    generator,
    kp_detector,
    relative=True)

# 비디오 저장
imageio.mimsave('../generated_video.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

# 노트북 출력
HTML(display_func(source_image, driving_video, predictions).to_html5_video())

100%|████████████████████████████████████████████████████████████████████████████████| 836/836 [02:49<00:00,  4.93it/s]
